# IMPORT MODULES

In [ ]:
import pandas as pd
import pandas_datareader.data as pdr
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import requests
import json

# SOME ISSUES WITH IMPORTING DATA

In [ ]:
response = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=demo")
alphadict = json.loads(response.text)
alphadict.keys()

In [ ]:
stock = pd.DataFrame(alphadict['Time Series (Daily)']).T
stock.info()

In [ ]:
type(stock.index)

In [ ]:
stock.index = pd.to_datetime(stock.index)
type(stock.index)

In [ ]:
stock['close']

In [ ]:
stock.columns

In [ ]:
stock.columns = ['open', 'high', 'low', 'close', 'volume']
stock.columns

In [ ]:
stock['close'].plot(figsize=(20,5), title='Microsoft (MSFT) daily closing prices'), plt.show();

In [ ]:
type(stock['close'][1])

In [ ]:
stock = stock.astype(float)
type(stock['close'][1])

# SAMPLE DATA

In [ ]:
sample = stock.sample(10)
sample

# SUMMARIZE DATA

In [ ]:
summary = stock.describe()
summary

# VISUALIZE DATA

In [ ]:
stock['close'].plot(figsize=(20,5), title='Microsoft (MSFT) daily closing prices'), plt.show();

# EXPLORING SIGNALS

In [ ]:
stock['SMA10'] = stock['close'].rolling(10).mean()
stock['SMA20'] = stock['close'].rolling(20).mean()
stock[['close','SMA10', 'SMA20']].plot(figsize=(20,5), grid = True, title='The 10 and 20 day simple moving averages of Microsoft'), plt.show();

# **SOME MORE ISSUES WITH IMPORTING DATA**

In [ ]:
stock.head()

In [ ]:
stock = stock.sort_index(ascending = True)
stock.head()

In [ ]:
stock['SMA10'] = stock['close'].rolling(10).mean()
stock['SMA20'] = stock['close'].rolling(20).mean()
stock[['close','SMA10', 'SMA20']].plot(figsize=(20,5), grid = True, title='The 10 and 20 day simple moving averages of Microsoft'), plt.show();

In [ ]:
stock.head()

In [ ]:
stock.dropna(inplace=True)
print(stock.head())
stock[['close','SMA10', 'SMA20']].plot(figsize=(20,5), grid = True, title='The 10 and 20 day simple moving averages of Microsoft'), plt.show();

#EVALUATING SIGNALS

In [ ]:
signal = pd.DataFrame()
signal['crossover']= stock['SMA10'] - stock['SMA20']
signal.tail()

In [ ]:
stock = stock.join(signal, how='left')
stock.tail()

In [ ]:
signal['close-close'] = stock['close'].diff()
signal.head()

In [ ]:
signal.dropna(inplace=True)
quality = (signal['crossover'] > 0) & (signal['close-close'] > 0) | (signal['crossover'] < 0) & (signal['close-close'] < 0)
quality.head()

In [ ]:
quality = quality.astype(int)
quality.head()

In [ ]:
proportion_correct = quality.sum()/quality.count()
proportion_correct

In [ ]:
information_coefficient = 2*proportion_correct - 1
information_coefficient

In [ ]:
signal.corr(method='spearman')